Caitlin Lindsay

In [10]:
!pip install kmodes
!pip install ucimlrepo
!pip install --upgrade scikit-learn

In [22]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.metrics.cluster import fowlkes_mallows_score

In [6]:
# fetch dataset 
soybean_small = fetch_ucirepo(id=91) 
  
# data (as pandas dataframes) 
X_soy = soybean_small.data.features 
y_soy = soybean_small.data.targets 
  
# metadata 
print(soybean_small.metadata) 
  
# variable information 
print(soybean_small.variables)

{'uci_id': 91, 'name': 'Soybean (Small)', 'repository_url': 'https://archive.ics.uci.edu/dataset/91/soybean+small', 'data_url': 'https://archive.ics.uci.edu/static/public/91/data.csv', 'abstract': "Michalski's famous soybean disease database", 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 47, 'num_features': 35, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1980, 'last_updated': 'Mon Feb 26 2024', 'dataset_doi': '10.24432/C5DS3P', 'creators': ['R. Michalski'], 'intro_paper': None, 'additional_info': {'summary': 'A small subset of the original soybean database.  See the reference for Fisher and Schlimmer in soybean-large.names for more information.\r\n    \r\nSteven Souders wrote:\r\n\r\n    > Figure 15 in the Michalski and Stepp paper (PAMI-82) says that the\r\n    > discriminant values for the

In [18]:
#BMM - EM
from scipy.special import logsumexp

class BernoulliMixture:
    
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
    
    def fit(self, x):
    self.x = x
    self.init_params()
    self.sk = np.zeros((self.n_components, self.x.shape[1]))
    self.pi_k = np.zeros(self.n_components)
    log_bernoullis = self.get_log_bernoullis(self.x)
    self.old_logL = self.get_log_likelihood(log_bernoullis)
    
    # Initially remember parameters after initialization
    self.remember_params()

    for step in range(self.max_iter):
        indices = np.random.choice(self.n_samples, self.batch_size, replace=False)
        x_batch = self.x[indices]
        log_bernoullis_batch = self.get_log_bernoullis(x_batch)

        # E-Step: Calculate responsibilities for the batch
        gamma_batch = self.get_responsibilities(log_bernoullis_batch)

        # Simulation of latent variables (gamma)
        latent_samples = np.array([np.random.choice(self.n_components, p=gamma) for gamma in gamma_batch])

        # Convert latent_samples to a format suitable for updating sk
        latent_samples_onehot = np.zeros((self.batch_size, self.n_components))
        latent_samples_onehot[np.arange(self.batch_size), latent_samples] = 1

        # Stochastic Approximation: Update the sufficient statistics
        step_size = 1.0 / (step + 1)
        self.sk = (1 - step_size) * self.sk + step_size * np.dot(latent_samples_onehot.T, x_batch) / self.batch_size
        self.pi_k = (1 - step_size) * self.pi_k + step_size * np.mean(latent_samples_onehot, axis=0)

        # Remember old parameters before updating
        self.remember_params()

        # M-Step: Update parameters based on sufficient statistics
        self.mu = self.sk / np.sum(self.sk, axis=1, keepdims=True)
        self.pi = self.pi_k / np.sum(self.pi_k)

        # Compute new log_likelihood:
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        if np.isnan(self.logL):
            self.reset_params()
            print(self.logL)
            break


    def remember_params(self):
    self.old_mu = self.mu.copy() if hasattr(self, 'mu') else None
    self.old_pi = self.pi.copy() if hasattr(self, 'pi') else None

    def reset_params(self):
        if hasattr(self, 'old_mu') and self.old_mu is not None:
            self.mu = self.old_mu.copy()
        if hasattr(self, 'old_pi') and self.old_pi is not None:
            self.pi = self.old_pi.copy()
        # Ensure the rest of the reset logic uses the existing parameters if old parameters were not set
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)

    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        mu_place = np.where(np.max(mu, axis=0) <= 1e-15, 1e-15, mu)
        return np.tensordot(x, np.log(mu_place), (1,1))
        
    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

IndentationError: expected an indented block after function definition on line 11 (660979022.py, line 12)

In [24]:
#SAEM
import numpy as np
from scipy.special import logsumexp

class BernoulliMixtureSAEM:
    def __init__(self, n_components, max_iter=100, batch_size=10):
        self.n_components = n_components
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.pi = None
        self.mu = None

    def init_params(self, X):
        n_samples, n_features = X.shape
        self.mu = np.random.rand(self.n_components, n_features)
        self.pi = np.ones(self.n_components) / self.n_components

    def get_log_bernoullis(self, X):
        # Bernoulli log probability calculations
        log_mu = np.log(self.mu.clip(min=1e-15))
        log_mu_neg = np.log((1 - self.mu).clip(min=1e-15))
        return np.dot(X, log_mu.T) + np.dot(1 - X, log_mu_neg.T)

    def get_responsibilities(self, log_bernoullis):
        weighted_log_probs = log_bernoullis + np.log(self.pi)
        log_sum = logsumexp(weighted_log_probs, axis=1, keepdims=True)
        return np.exp(weighted_log_probs - log_sum)

    def fit(self, X):
        self.init_params(X)
        n_samples = X.shape[0]

        for step in range(self.max_iter):
            indices = np.random.choice(n_samples, self.batch_size, replace=False)
            X_batch = X[indices]

            log_bernoullis_batch = self.get_log_bernoullis(X_batch)
            gamma = self.get_responsibilities(log_bernoullis_batch)

            # Update step for mu and pi
            gamma_sum = gamma.sum(axis=0)
            self.mu = (gamma.T @ X_batch + self.mu * self.batch_size) / (gamma_sum[:, None] + self.batch_size)
            self.pi = (gamma_sum / self.batch_size + self.pi * self.batch_size) / (self.batch_size + self.batch_size)

    def predict(self, X):
        log_bernoullis = self.get_log_bernoullis(X)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)



In [38]:
encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(X_soy)

model = BernoulliMixtureSAEM(n_components=4, max_iter=100, batch_size=10)
model.fit(X_encoded)
clusters = model.predict(X_encoded)
print(clusters)

print("Cluster centers (mu parameters):")
print(model.mu)

fmi_bmm = fowlkes_mallows_score(y_series, clusters)
ari_bmm = adjusted_rand_score(y_series, clusters)
nmi_bmm = normalized_mutual_info_score(y_series, clusters)

print("FMI (BMM):", fmi_bmm)
print("ARI (BMM):", ari_bmm)
print("NMI (BMM):", nmi_bmm)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1]
Cluster centers (mu parameters):
[[4.95975346e-01 8.58874002e-01 7.14049119e-01 9.50392060e-01
  5.29056160e-01 9.79132103e-01 1.05741280e-02 4.87740659e-01
  9.74325073e-01 9.38236289e-01 5.15958914e-02 7.53812128e-01
  6.20036626e-01 7.77711298e-01 7.26379343e-01 9.56513482e-01
  6.88606818e-02 5.21416954e-01 8.78408721e-01 7.77037638e-01
  2.04489279e-01 7.17924015e-01 8.56512349e-01 5.23707958e-01
  6.75940013e-01 8.75693551e-01 4.32607483e-01 6.19614816e-02
  4.13314949e-01 4.32884948e-01 3.06037637e-01 1.95669370e-01
  6.51525913e-01 5.00573565e-01 9.37197718e-01 9.29361539e-01
  3.02807337e-01 6.93988059e-01 2.29117174e-01 2.04343988e-01
  7.85112987e-01 3.02782434e-02 3.08254902e-01 6.45801267e-01
  2.64989163e-01 3.67378296e-01 1.62182855e-02 5.62557196e-01
  2.15629114e-01 5.61472042e-01 4.23663595e-01 1.84272998e-01
  6.42196235e-01 8.71661296e-01 9.98167501e-01 1.19495639e-01
  

In [31]:
from kmodes.kmodes import KModes

# Perform KModes clustering
km = KModes(n_clusters=4)  
clusters_km = km.fit_predict(X_soy)
y_series = y_soy.squeeze()
# Compute FMI
fmi_km = fowlkes_mallows_score(y_series, clusters_km)
print("FMI (KModes):", fmi_km)

# Compute ARI
ari_km = adjusted_rand_score(y_series, clusters_km)
print("ARI (KModes):", ari_km)

# Compute NMI
nmi_km = normalized_mutual_info_score(y_series, clusters_km)
print("NMI (KModes):", nmi_km)

FMI (KModes): 1.0
ARI (KModes): 1.0
NMI (KModes): 0.9999999999999999


$\textbf{Report}$<br>

There is a big gap between the BMM SAEM and K-Mode Results. 
This might be because K-Mode is designed for categorical data, which is the type of data in the soybean dataset.
The dataset had to be preprocessed and one-hot encoded before performing BMM SAEM on it. Therefore, while K-Mode
could naturally identify the patterns in the data, transorming the categorical data to binary format with one-hot encoding could have diluted some relationships. 

Additionally, the randomness in SAEM can lead to different optima, and can change the results. However, trying out 3,5,6,and 8
number of components lead to lower or around the same result. Furthermore, FMI  being 1 in K-Modes shows that it identified all true positive pairs without any false positives or negatives. For ARI and NMI, they might be lower as they are sensitive to matching clusters to the ground truth. So, a lower score may mean that the BMM clusters have overlaps. And as ARI and NMI are more focused on mathcing the clusters versus the ground-truth clusters and FMI focuses more on the mean of precision and recal rather than overall cluster comparison, FMI has a higher value because many pairs of points that belong together are still in the same cluster.